In [246]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [247]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [248]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [249]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [250]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [251]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [252]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [253]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((31095723290840840374048580348366846963100248920115384244364807885837456321768444392091534373326861214561127543061280121255126431120784979912949903773959233358792739512152324235006731030119229928360252736987219511810111885432656920045133407608156128926113500285146458722570436633568190432899044521662709083068413975491349923993867003709642515077929403961955818336422030707195811394976449403022497192274476173831527617584859820792018317615785031026199894303301688203999471817920243689894563145588779661491788391444448184846020766797696142681438885914808606737748424701279258966151306152199184816641327081188135280577393, 3600241420679714126188645504835319519084273597601222931976716203319753239104115142777629476220021286197199765830320910334896249094541314454529456299060026313499518532846330165580386183384826275709580940974598525522590811725850994351596813927742484321336107205817367728982371917809931421754289460144830185310848917902506083802173878721088822230077406385804410172618456694470878858

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [254]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [255]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [256]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

5088058972943892628749614998296233532632680103800239141074178988613446380298559066302521006841755600585429898455582671829233194660814057226865639397453203414208337565428027359648216756254337926579158863598355063358287008751677513673846753144807167770467288845560628613537618707686182966987276009824819403508877343466505894427409730571082637773299703398809847558411217916218089021012224153469418859461492741782806800626101397798967401048384909882187688177506493277083560215258337569553673010558818197675008246172961929536965573518515684131178006335249602079103165456663175597244843302834741584990956624988274363285686

In [257]:
y_hack = pow(x_hack, e, n)
y_hack

21357766653201511729131705772081165907486154528470305333208380483191592583052673971570818664388160141120670580704916662969957534800368729197776897840701740800317744482304622958125095208887074393353367874446444423395852571355982382775589224345976936719066703664430223387788359571810052106881177165537067666670724524945731118640693471998025703008206035915316754666083740548195482916406909544802177973516501712270662693384912834549898597115359157733540943125776336310708279140289006057523555174837096746409647122664165044548383329819601247003256218311803444920091007834153005063125025828012304870189155454759593934561812

# Hash

In [258]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [259]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

5881510139291921089108565814727078934368849797613795399315423341039620776655234377737071046236398955309311730186584780635492362342052531210106961359184748226484851393635084462114111085080496545785794350512956833478054779944515285319545989573185319664696088144885322361173039794928462943851685843014248824182423131993042085879069557195978673457298543864297106646188659958093728988136730533825222234167330472844766822458815312607522240284674333326008180437198568369461421824443869245733419864349090311462415197106808526154387514942808766797316506660967853573548442974405401130455399925900067008474285397325736407068221

# Decryption of hash (sign) with Sender pub key

In [260]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [261]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

2340509926146499719798124187838309

# Hash verification 

In [262]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(int.from_bytes(final_h.digest()) == decrypt_to_hash)
#print(final_h.digest() == decrypt_to_hash)

True


# Get plaintext

In [263]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

In [264]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Generating keypairs
sender_keypair = generate_keypair() #generate sender keypair
receiver_keypair = generate_keypair() #generate receiver keypair
print(sender_keypair, '\n', receiver_keypair)

#Define message x
msg = 'secret message'.encode('utf-8') #secret message
x = int.from_bytes(msg, 'big') #turn bytes to integer

# Encrypt message x with receiver public key -> ciphertext y
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n) 

# Compute hash
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

#Signing 
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n) #signing hash with sender private key

((28486450626734503217448576473681531240199817672921838533960382402634245117395135628982511120335168143867109975210286658291193144759369736753757899388474857448392360150037153669886384521740728163890995454298968497788846648393263330684540645871251944562733969953569095171314621318881866274553413988380946646860670611490065833271017993746875251576960135896726581131782567424465104321952869901243273159005050967187580487565065209836921481778714326578051702661623284654704332571119293898446056135205154238830220747748110699804992915202038932979992998568231727847679880624892467678766516430253760363714099001461285481479107, 9400770064662337294875771962617734984539105785285962524066133504546058988877656351618108850589616123134651181652091708124907887782761514737530785369655969483883726618657340777820506594144784009991919544197241130296989768844236977861139546526642241861863101627466620678418859703868069671579160347326302086896623468383111135028726614137614532338925385488418487274477680687625881486

In [265]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)

n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)

msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
final_h_int = int.from_bytes(final_h.digest(), 'big')  

valid_signature = decrypt_to_hash == final_h_int
print("Signature valid?", valid_signature)

decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

Signature valid? True


'secret message'

In [266]:
from Crypto.Cipher import DES3, AES
from Crypto.Random import get_random_bytes

def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())


3DES key (hex): b97557d0abc483109d7c9de6e9a2ad0d627acbc879fbefc2
K1: b97557d0abc48310
K2: 9d7c9de6e9a2ad0d
K3: 627acbc879fbefc2


In [267]:
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

In [268]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)

In [269]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        #pt = unpad(cipher.decrypt(ct), DES3.block_size)
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [270]:
decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
